In [288]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares


In [289]:
data = pd.read_excel('test.xlsx')
drop_num = [] # data preprocessing, remove odd data

for i, val in enumerate(data['I']):
    if (i>0) and (data['V_T'][i-1] > data['V_T'][i]) == 1:
        drop_num.append(i)

    if (val < 0) or (val > 9E-6):
        drop_num.append(i)
data.drop(drop_num, inplace=True)

In [290]:
groups = data.groupby("cell No.")
pieces = [group for _, group in groups]
splitted_pieces = np.array_split(pieces, 6)

c:\Users\nmdl-hspice\anaconda3\envs\eqprop_python\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [291]:
def model_function(x, params):
    # Example function: a * x^2 + b * x + c
    Is, n = params
    return Is*(np.exp(x/n*0.026)-1)

def residual_function(params, x, y):
    return model_function(x, params) - y

result_Is = []
result_n = []
for i, piece in enumerate(splitted_pieces) :
    x = splitted_pieces[i][0]['V_T']
    y = splitted_pieces[i][0]['I']
    initial_params = [1e-9, 1]
    result = least_squares(residual_function, initial_params, args=(x,y), method='lm')
    result_Is.append(result.x[0])
    result_n.append(result.x[1])

In [292]:
print(np.average(result_Is))
print(np.average(result_n))

2.8185369492738913e-09
0.01844628009938582
